In [3]:
import pandas as pd
import numpy as np
import requests, json
import os

In [4]:
client_id = os.environ['BNET_CLIENT_ID']
client_secret = os.environ['BNET_SECRET']

def create_access_token(client_id, client_secret, region = 'us'):
    data = { 'grant_type': 'client_credentials' }
    response = requests.post('https://%s.battle.net/oauth/token' % region, data=data, auth=(client_id, client_secret))
    return response.json()

token = create_access_token(client_id, client_secret)['access_token']

# Member List

In [5]:
members = requests.get(f'https://us.api.blizzard.com/data/wow/guild/barthilas/steamed-hams/roster?namespace=profile-us&locale=en_US&access_token={token}').json()

char_id = [i['character']['id'] for i in members['members']]
char_name = [i['character']['name'].lower() for i in members['members']]
char_level = [i['character']['level'] for i in members['members']]
char_class = [i['character']['playable_class']['id'] for i in members['members']]

members_df = pd.DataFrame(
    {'char_id':char_id,
     'char_name':char_name,
     'char_level':char_level,
     'char_class':char_class}
).set_index('char_name')
members_df = members_df[members_df['char_level'] == 70].sort_values('char_name')
members_df.head()


,char_id,char_level,char_class
char_name,,,
agnikai,203257707,70,9
ahmanva,181697502,70,9
antremodes,180625739,70,1
anxti,203254720,70,3
beardzdk,203791547,70,6


## Get current active members

In [6]:
time_list = []

for i in members_df.index:
    req = requests.get(f'https://us.api.blizzard.com/profile/wow/character/barthilas/{i}?namespace=profile-us&locale=en_US&access_token={token}').json()
    ts = req.get('last_login_timestamp')
    
    time_list.append(ts)
    
members_df['last_login'] = [i / 1000 for i in time_list]
members_df['last_login'] = pd.to_datetime(members_df['last_login'], unit='s')
members_df['date_diff'] = members_df.last_login - pd.Timestamp.now()

members_df['active_30_days'] = [i.days >= -30 for i in members_df.date_diff]
members_df['active_60_days'] = [i.days >= -60 for i in members_df.date_diff]
members_df

,char_id,char_level,char_class,last_login,date_diff,active_30_days,active_60_days
char_name,,,,,,,
agnikai,203257707,70,9,2023-06-22 12:09:24,-147 days +13:24:04.481541,False,False
ahmanva,181697502,70,9,2023-02-18 10:54:25,-271 days +12:09:05.481541,False,False
antremodes,180625739,70,1,2023-02-08 21:40:45,-281 days +22:55:25.481541,False,False
anxti,203254720,70,3,2023-04-03 11:17:22,-227 days +12:32:02.481541,False,False
beardzdk,203791547,70,6,2023-11-14 11:44:13,-2 days +12:58:53.481541,True,True
...,...,...,...,...,...,...,...
zabu,178579839,70,1,2023-08-14 15:38:26,-94 days +16:53:06.481541,False,False
zymbodia,201607820,70,1,2023-10-19 09:25:39,-28 days +10:40:19.481541,True,True
zymss,198791240,70,5,2023-11-06 15:09:55,-10 days +16:24:35.481541,True,True


# Subset to currently active (30/60/etc. days)

In [7]:
members_df = members_df[members_df['active_30_days']]
members_df.to_csv('steamed_hams.csv')
members_df

,char_id,char_level,char_class,last_login,date_diff,active_30_days,active_60_days
char_name,,,,,,,
beardzdk,203791547,70,6,2023-11-14 11:44:13,-2 days +12:58:53.481541,True,True
beardzz,203426723,70,2,2023-11-10 09:57:34,-6 days +11:12:14.481541,True,True
beheadme,202008718,70,1,2023-11-05 12:01:11,-11 days +13:15:51.481541,True,True
bellfulgur,133925404,70,7,2023-11-15 13:37:56,-1 days +14:52:36.481541,True,True
blessedthot,202005152,70,2,2023-11-14 10:48:03,-2 days +12:02:43.481541,True,True
...,...,...,...,...,...,...,...
yenoslas,184606459,70,7,2023-11-02 13:47:22,-14 days +15:02:02.481541,True,True
zymbodia,201607820,70,1,2023-10-19 09:25:39,-28 days +10:40:19.481541,True,True
zymss,198791240,70,5,2023-11-06 15:09:55,-10 days +16:24:35.481541,True,True


# Add useful class, spec data
TODO: separate from here onwards from the initial members extraction

In [8]:
members_df =pd.read_csv('steamed_hams.csv').set_index('char_name')
members_df

,char_id,char_level,char_class,last_login,date_diff,active_30_days,active_60_days
char_name,,,,,,,
beardzdk,203791547,70,6,2023-11-14 11:44:13,-2 days +12:58:53.481541,True,True
beardzz,203426723,70,2,2023-11-10 09:57:34,-6 days +11:12:14.481541,True,True
beheadme,202008718,70,1,2023-11-05 12:01:11,-11 days +13:15:51.481541,True,True
bellfulgur,133925404,70,7,2023-11-15 13:37:56,-1 days +14:52:36.481541,True,True
blessedthot,202005152,70,2,2023-11-14 10:48:03,-2 days +12:02:43.481541,True,True
...,...,...,...,...,...,...,...
yenoslas,184606459,70,7,2023-11-02 13:47:22,-14 days +15:02:02.481541,True,True
zymbodia,201607820,70,1,2023-10-19 09:25:39,-28 days +10:40:19.481541,True,True
zymss,198791240,70,5,2023-11-06 15:09:55,-10 days +16:24:35.481541,True,True


In [9]:
# Add Scores
def rank_set(member_list):

    rank_list = {}

    for i in member_list:

        req = requests.get(f"https://raider.io/api/v1/characters/profile?region=us&realm=barthilas&name={i}&fields=mythic_plus_scores_by_season:current").json()

        if 'mythic_plus_scores_by_season' in req:
            
            char_name = req['name']
            char_class = req['class']
            scores = req['mythic_plus_scores_by_season'][0]['scores']
                
            rank_list[(char_name,char_class,'spec_0')] = {'score':scores['spec_0']}
            rank_list[(char_name,char_class,'spec_1')] = {'score':scores['spec_1']}
            rank_list[(char_name,char_class,'spec_2')] = {'score':scores['spec_2']}  
            rank_list[(char_name,char_class,'spec_3')] = {'score':scores['spec_3']}
                          
            rank_list[(char_name,char_class,'tank')] = {'score':scores['tank']}
            rank_list[(char_name,char_class,'healer')] = {'score':scores['healer']}
            rank_list[(char_name,char_class,'dps')] = {'score':scores['dps']}
            
            #Re-incorporate tank/healer/dps

    return rank_list

ham_scores = rank_set(members_df.index)
ham_scores

{('Beardzdk', 'Death Knight', 'spec_0'): {'score': 0},
 ('Beardzdk', 'Death Knight', 'spec_1'): {'score': 0},
 ('Beardzdk', 'Death Knight', 'spec_2'): {'score': 0},
 ('Beardzdk', 'Death Knight', 'spec_3'): {'score': 0},
 ('Beardzdk', 'Death Knight', 'tank'): {'score': 0},
 ('Beardzdk', 'Death Knight', 'healer'): {'score': 0},
 ('Beardzdk', 'Death Knight', 'dps'): {'score': 0},
 ('Beardzz', 'Paladin', 'spec_0'): {'score': 0},
 ('Beardzz', 'Paladin', 'spec_1'): {'score': 0},
 ('Beardzz', 'Paladin', 'spec_2'): {'score': 0},
 ('Beardzz', 'Paladin', 'spec_3'): {'score': 0},
 ('Beardzz', 'Paladin', 'tank'): {'score': 0},
 ('Beardzz', 'Paladin', 'healer'): {'score': 0},
 ('Beardzz', 'Paladin', 'dps'): {'score': 0},
 ('Beheadme', 'Warrior', 'spec_0'): {'score': 0},
 ('Beheadme', 'Warrior', 'spec_1'): {'score': 0},
 ('Beheadme', 'Warrior', 'spec_2'): {'score': 0},
 ('Beheadme', 'Warrior', 'spec_3'): {'score': 0},
 ('Beheadme', 'Warrior', 'tank'): {'score': 0},
 ('Beheadme', 'Warrior', 'healer')

In [10]:
ham_scores_df = pd.DataFrame(ham_scores).T.reset_index()
ham_scores_df.columns = ['char_name','class','spec_n','score']
ham_scores_df = ham_scores_df.set_index(['char_name','class','spec_n'])
ham_scores_df

score
char_name class        spec_n       
Beardzdk  Death Knight spec_0    0.0
                       spec_1    0.0
                       spec_2    0.0
                       spec_3    0.0
                       tank      0.0
...                              ...
Zymzz     Monk         spec_2    0.0
                       spec_3    0.0
                       tank      0.0
                       healer    0.0
                       dps       0.0

[623 rows x 1 columns]

In [11]:
# Clarify Class and Spec
spec_df = pd.read_csv('specs.csv').set_index(['class','spec_n'])
spec_df

spec_name spec_short2 spec_short
class        spec_n                                      
Death Knight spec_0          Blood         BLD      Blood
             spec_1          Frost        FRST      Frost
             spec_2         Unholy         UNH     Unholy
Demon Hunter spec_0          Havoc         HVC      Havoc
             spec_1      Vengeance         VDH        VDH
Druid        spec_0        Balance         BAL    Balance
             spec_1          Feral         FER      Feral
             spec_2       Guardian         GUA       Bear
             spec_3    Restoration       RESTO      Resto
Hunter       spec_0  Beast Mastery          Bm         BM
             spec_1   Marksmanship          MM         MM
             spec_2       Survival        SURV       Surv
Mage         spec_0         Arcane         ARC     Arcane
             spec_1           Fire        FIRE       Fire
             spec_2          Frost        FRST      Frost
Monk         spec_0     Brewmaster        BREW       Brew
             spec_1     Mistweaver        MIST       Mist
             spec_2     Windwalker          WW         WW
Paladin      spec_0           Holy        HPAL       HPal
             spec_1     Protection        PROT       Prot
             spec_2    Retribution         RET        Ret
Priest       spec_0     Discipline        DISC       Disc
             spec_1           Holy         HPR       Holy
             spec_2         Shadow        SHAD     Shadow
Rogue        spec_0  Assassination         SIN        Sin
             spec_1         Outlaw        OUTL     Outlaw
             spec_2       Subtlety         SUB        Sub
Shaman       spec_0      Elemental         ELE        Ele
             spec_1    Enhancement         ENH        Enh
             spec_2    Restoration       RESTO      Resto
Warlock      spec_0     Affliction        AFFL       Affl
             spec_1     Demonology        DEMO       Demo
             spec_2    Destruction       DSTRO     Destro
Warrior      spec_0           Arms        ARMS       Arms
             spec_1           Fury        FURY       Fury
             spec_2     Protection        PROT       Prot
Evoker       spec_0    Devastation       DVOKE    Devoker
             spec_1   Preservation       PVOKE   Prevoker
             spec_2   Augmentation         AUG   Augvoker

In [12]:
df = ham_scores_df.join(spec_df).dropna().reset_index()
df

,class,spec_n,char_name,score,spec_name,spec_short2,spec_short
0,Death Knight,spec_0,Beardzdk,0.0,Blood,BLD,Blood
1,Death Knight,spec_0,Dcfnight,0.0,Blood,BLD,Blood
2,Death Knight,spec_0,Dingplague,0.0,Blood,BLD,Blood
3,Death Knight,spec_0,Eesod,668.6,Blood,BLD,Blood
4,Death Knight,spec_0,Fannybaws,0.0,Blood,BLD,Blood
...,...,...,...,...,...,...,...
261,Warrior,spec_2,Dtx,0.0,Protection,PROT,Prot
262,Warrior,spec_2,Extraineous,0.0,Protection,PROT,Prot
263,Warrior,spec_2,Fcw,0.0,Protection,PROT,Prot
264,Warrior,spec_2,Jackrum,0.0,Protection,PROT,Prot


# Export

In [13]:
df.to_csv('Steamed Hams M+ Scores.csv')